In [1]:
# load the vector files
import sys
import io
setting = "p2v"
Dataset = "pubmed"

vectorFilesDir = "../Data/"+Dataset+"/vectors/"+setting+"/"+setting+".txt"
allPaperVectors = []

with open(vectorFilesDir, 'r', encoding = 'utf8') as f:
    for line in f:
        read_data = line.split(" ")
        paper_Vectors = read_data
        allPaperVectors.append(paper_Vectors)
f.close()
        
print("Total vector records:",len(allPaperVectors))
print(allPaperVectors[0])

Total vector records: 3149758
['22516865', '0.0422827', '-0.0443959', '-0.00631522', '-0.0297457', '-0.203585', '0.0791923', '-0.156993', '0.102914', '-0.112826', '0.0301516', '-0.118142', '-0.0447324', '0.0131143', '0.127706', '0.107417', '-0.0843709', '-0.0808756', '-0.0504176', '0.132817', '0.0157529', '-0.0383218', '0.165365', '-0.0619297', '-0.0345336', '0.110633', '0.0860297', '-0.0835764', '0.193302', '0.015641', '0.0323645', '0.00893256', '0.151895', '0.0534986', '-0.18639', '-0.0558457', '-0.0603591', '-0.00530632', '-0.000749852', '0.0772172', '0.0646557', '-0.106445', '-0.028214', '0.0400324', '0.11686', '-0.153841', '-0.064435', '-0.0270715', '-0.0234374', '0.0588498', '-0.0750828', '-0.107259', '-0.0351021', '-0.0166648', '-0.135551', '0.1277', '-0.118618', '-0.0417162', '0.0543777', '0.129501', '-0.152003', '0.0624739', '0.175164', '0.0680417', '-0.0896099', '0.171857', '-0.0123408', '-0.0424403', '-0.017759', '0.0631562', '-0.0947245', '-0.0297129', '-0.0952609', '0.0229

In [2]:
import re
import numpy as np
import os

# collect data
fileDir = "../Data/"+Dataset+"/filteredSameNameAuthor/filter=10/"
fileList = os.listdir(fileDir)
fileList.sort()
print(fileList)

['alfredo martinez.txt', 'alfredo martinez0.txt', 'alfredo martinez1.txt', 'amit patel.txt', 'amit patel0.txt', 'amit patel1.txt', 'ana castro.txt', 'ana castro0.txt', 'ana castro1.txt', 'ana castro2.txt', 'anna ferrari.txt', 'anna ferrari0.txt', 'anna ferrari1.txt', 'bin liu.txt', 'bin liu0.txt', 'bin liu1.txt', 'carmen moreno.txt', 'carmen moreno0.txt', 'carmen moreno1.txt', 'carmen torres.txt', 'carmen torres0.txt', 'carmen torres1.txt', 'chao liu.txt', 'chao liu0.txt', 'chao liu1.txt', 'cheng luo.txt', 'cheng luo0.txt', 'cheng luo1.txt', 'chung-may yang.txt', 'chung-may yang0.txt', 'chung-may yang1.txt', 'david g lloyd.txt', 'david g lloyd0.txt', 'david g lloyd1.txt', 'fang liu.txt', 'fang liu0.txt', 'fang liu1.txt', 'feng liu.txt', 'feng liu0.txt', 'feng liu1.txt', 'feng xu.txt', 'feng xu0.txt', 'feng xu1.txt', 'francisco esteves.txt', 'francisco esteves0.txt', 'francisco esteves1.txt', 'francisco j blanco.txt', 'francisco j blanco0.txt', 'francisco j blanco1.txt', 'giovanni volpe

In [3]:
# remove author(positive sample) from other(negative sample)
import random
def extractNegativeSample(positiveSample, allSample):
    negativeSample = [x for x in allSample if x not in positiveSample]
    print("Total negative sample size:", len(negativeSample))
    return negativeSample

In [4]:
# collect class vectors
import pandas as pd
import numpy as np

def extractVectors(author_pids, NegativeSample_pid, allPaperVectors):
    # extract class one vectors
    author_features = []
    for pid in author_pids:
         for paper_Vectors in allPaperVectors:
            if(paper_Vectors[0] == pid):
                author_features.append(paper_Vectors)
    print("Positive sample size: ", len(author_features))
    classOne = pd.DataFrame(author_features)
    classOne["label"] = 0
    # extract class two vectors
    other_features = []
    for pid in NegativeSample_pid:
        for paper_Vectors in allPaperVectors:
            if(paper_Vectors[0] == pid):
                other_features.append(paper_Vectors)
    print("Negative sample size: ", len(other_features))
    classTwo = pd.DataFrame(other_features)
    classTwo["label"] = 1
    return classOne, classTwo


In [5]:
# combine data from different class get all data
def combineClassesData(classOne,classTwo):
    combinedData = pd.concat([classOne, classTwo])
    combinedData = combinedData.sample(frac=1).reset_index(drop=True)
    # take the paper id out
    paperID = combinedData[0]
    # split data and label
    data = combinedData.drop([0,'label'], axis=1)
    label = combinedData['label']
    print("Total sample size and shape: ",data.shape)
    return data, label, paperID

In [6]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import (precision_score, recall_score,f1_score,accuracy_score)
# cross validation
def k_fold_cv(author, data, label, paperID, classifier, k=10):
    kf = KFold(n_splits=k, shuffle=False)
    allTrueLabel = []
    allPredLabel = []
    for counter,(train_index, test_index) in enumerate(kf.split(data)):
        # print("TRAIN:", train_index, " \n TEST:", test_index)
        # split train and test
        data_train, data_test = data.iloc[train_index], data.iloc[test_index]
        label_train, test_true_label = label.iloc[train_index], label.iloc[test_index]
        # fit data to svm
        classifier.fit(data_train, label_train)
        # get predicted label
        label_pred = classifier.predict(data_test)
        allTrueLabel.extend(test_true_label)
        allPredLabel.extend(label_pred)
#         # get predict proba
#         proba = classifier.predict_proba(data_test)
        # find out which sample cause the issue
        print("Pred: ",label_pred)
        print("True: ", test_true_label.values.tolist())
        print("Mislabeled sample: ",end='')
        for i in range(len(test_true_label)):
            if(label_pred[i]!=test_true_label[test_index[i]]):
                print(paperID[test_index[i]]+",",end='')
        print()
        # print("True positive: {tp}, False positive: {fp}, False negative: {fn}, True negative: {tn}"
        # .format(tp=round_tp, fp=round_fp, fn=round_fn, tn=round_tn))

    accuracy = accuracy_score(allTrueLabel, allPredLabel)
    f1 = f1_score(allTrueLabel, allPredLabel,average='binary')
    precision = precision_score(allTrueLabel, allPredLabel)
    recall = recall_score(allTrueLabel, allPredLabel)
    tn,fp,fn,tp = metrics.confusion_matrix(allTrueLabel, allPredLabel).ravel()
    
    print("Author: ", author)
    print("Classifier: ",classifier)
    print(metrics.classification_report(allTrueLabel, allPredLabel))
    print(metrics.confusion_matrix(allTrueLabel, allPredLabel).ravel())
    print("Accuracy: ",accuracy)
    print("F1: ", f1)
    print("Precision: ", precision)
    print("Recall: ", recall)
    
    return accuracy, f1, precision, recall, tn, fp, fn, tp
    # return ppv, npv, specificity, sensitivity, accuracy, f1proba = linear_svc.predict_proba(allDatas)

In [7]:
# hard code to read the file one by one
# store the features for classification
author_pids = []
other_pids = []
name = "lei wang0"
# author as positive sample, other as all samples
with open(fileDir+name+".txt", 'r', encoding = 'utf8') as f:
    for line in f:
        author_pids.extend(line.strip().split(" "))

with open(fileDir+"lei wang.txt", 'r', encoding = 'utf8') as f:
    for line in f:
        other_pids.extend(line.strip().split(" "))
        
# size of each class
print(len(author_pids))
print(len(other_pids))

16
150


In [8]:
# extract negative Sample
NegativeSample_pid = extractNegativeSample(author_pids, other_pids)
print("Choicen negative sample ", len(NegativeSample_pid))

Total negative sample size: 134
Choicen negative sample  134


In [9]:
classOne, classTwo = extractVectors(author_pids,NegativeSample_pid,allPaperVectors)
print(classOne.shape)
print(classTwo.shape)

Positive sample size:  16
Negative sample size:  134
(16, 102)
(134, 102)


In [10]:
data, label, pid = combineClassesData(classOne, classTwo)

Total sample size and shape:  (150, 100)


In [14]:
from sklearn import svm
# create linear SVM model
linear_svc = svm.SVC(kernel='linear', class_weight='balanced', probability=True,cache_size=4000)

accuracy, f1, precision, recall, tn, fp, fn, tp= k_fold_cv(name,data, label, pid, linear_svc,10)
print(linear_svc)

# get number of support vectors for each class
print(linear_svc.n_support_)

'''
# compute the distance to decision boundry (Not same as confidence measure)
Distance = linear_svc.decision_function(allDatas)

# computer the confidence measure (Platt scaling: transforming the outputs of a 
# classification model into a probability distribution over classes)
# P(class/input) = 1 / (1 + exp(A * f(input) + B))
# P(class/input) is the probability that “input” belongs to “class” 
# and f(input) is the signed distance of the input datapoint from the boundary,
# which is basically the output of “decision_function”. 

proba = linear_svc.predict_proba(allDatas)

'''

# create rbf SVM model with C=10 where (C*Error) is added into minimize function
# C big means error matter more
rbf_svc = svm.SVC(kernel='rbf',probability=True)

# fit model and do 10-fold cv
k_fold_cv(name,data, label, pid, rbf_svc, 10)

# get number of support vectors for each class
print(rbf_svc.n_support_)


Pred:  [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1]
True:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
Mislabeled sample: 
Pred:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
True:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Mislabeled sample: 
Pred:  [1 1 1 1 0 1 1 1 1 1 0 1 0 1 1]
True:  [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1]
Mislabeled sample: 
Pred:  [1 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
True:  [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Mislabeled sample: 
Pred:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
True:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
Mislabeled sample: 
Pred:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
True:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Mislabeled sample: 
Pred:  [1 1 1 1 1 0 1 0 0 1 1 1 1 1 1]
True:  [1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1]
Mislabeled sample: 
Pred:  [1 1 1 1 0 1 1 1 1 1 1 1 1 1 1]
True:  [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Mislabeled sample: 
Pred:  [1 1 1 1 1 1 0 1 0 1 1 1 1 1 0]
True:  [1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0]
Mislabeled s

/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
